We want to make use of the analytical solution for the Wasserstein distance in 1D. By projecting components of the posterior mixture onto their principle axes, the variables in the new basis become independent and therefore can be considered separately. The maps therefore must be chosen so that their projections onto these axes are mapped to the standard normal (1D). We could potentially do this by minimising the Wasserstein distance between them. The advantage of this is that we can rescale the two distributions arbitrarily so that they both some to one as discrete distributions as the scaling is absorbed into the proportionallity factor that we only care about at the end.

In [9]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
from scipy.linalg import fractional_matrix_power as fmp

from Core_Functions import gmm_base as gb
from Core_Functions import gmm_plot as gp

import ot

In [10]:
ref = multivariate_normal(mean = 0, cov = 1)

proj = multivariate_normal(mean = 2, cov = 0.5)

x = np.linspace(-5,5,30)

C = np.zeros([30,30])

for i in range(30):
    for j in range(30):
        C[i,j] = (x[i] - x[j])**2
        
f = ref.pdf(x)
f /= np.sum(f)
g = proj.pdf(x)
g /= np.sum(g)

ot.emd2(f,g,C)

4.107022139229891

In [ ]:
def Evaluate_Mixture(f,x):
    """
    Gives the likelihood at any given set of points in a Guassian mixture model.
    Inputs:
        f: mixture being evaluated (Gaussian_Mixture)
        x: points being evaluated at (array)
    """
    num_x, d = np.shape(x)
    
    #do check here that d = f.d, error if not
    fx = np.zeros([num_x])
    
    for i in range(f.n):
        fx += f.w[i] * multivariate_normal.pdf(x,mean=f.m[i,:],cov=f.cov[i,:,:])
    
    return fx

## Optimisation Problem

We have the goal of minimising the following expression:

\begin{equation}
M = \sum_{k=1}^m \mathcal{W}_2^2 (\phi, Q_k (\Lambda, \mathbf{b}))
\end{equation}

We therefore firstly need a way to compute the distributions $Q_k$ for any given inputs, then find the derivative of the Wasserstein distance with respect to these inputs. The distributions will be defined as follows.

\begin{equation}
Q_k(m) = \frac{\lambda_k}{s} L\circ \beta_k(m) h\circ \beta_k(m) C_k(m)
\end{equation}

This function $C_k$ is the classifier and in turn is defined as follows.

\begin{equation}
C_k(m) = \frac{h\circ \beta_k^{-1}(m)}{\sum_{j=1}^m q_j h\circ \beta_j^{-1}(m)}
\end{equation}

Each of these maps $\beta$ are defined by a scaling and shift.

\begin{equation}
\beta_k(m) = \lambda_k m + b_k
\end{equation}

There is the additional constraint on $\lambda_k$ that they must be positive to ensure the transport is optimal (gradient of a convex function). 

The final goal will be to compute

\begin{equation}
\frac{\partial M}{\partial \lambda_j}, \frac{\partial M}{\partial b_j}, \frac{\partial M}{\partial q_j}
\end{equation}

[WILL ONLY NEED TO KEEP SOME OF THIS]

\begin{equation}
\frac{\partial C_k}{\partial q_j} = -\left(\frac{h\circ\beta_j^{-1}(m)}{\sum_{l=1}^m q_l h\circ \beta_l^{-1}(m)}\right)^2 = -C_j(m)^2
\end{equation}

It is important to remember here that, by nature of the maps,

\begin{equation}
h\circ\beta_k^{-1}(m) = \phi(m;b_k,\lambda_k^2)
\end{equation}

If $\sigma_k^2$ is the variance of the $k$-th component, then

\begin{align}
\frac{\partial \sigma_k^2}{\partial \lambda_k} &= 2\lambda_k \\
\frac{\partial \sigma_k^2}{\partial b_k} &= 0
\end{align}

As for the mean $\mu_k$,

\begin{align}
\frac{\partial \mu_k}{\partial \lambda_k} &= 0 \\
\frac{\partial \mu_k}{\partial b_k} &= 1
\end{align}

These must then be linked to the change in sample heights. [HAVE THESE IN MY BOOK SOMEWHERE]

\begin{align}
\frac{\partial \phi}{\partial q_j} &= 0\\
\frac{\partial \phi}{\partial \mu_j} &= \frac{m - \mu_j}{\sigma_j}\phi_j \\
\frac{\partial \phi}{\partial \sigma_j^2} &= \left[ \frac{(m-\mu_j)^2}{\sigma_j^3} - \sigma_j \right] \phi_j
\end{align}

We will then need the derivatives of the 2-Wasserstein distance with respect to mass changes and position shifts. [HAD BOTH OF THESE FOR ENTROPIC REGULARISATION SO SHOULD HAVE EQUIVALENT FOR ANALYTICAL SOLUTION HERE TOO]

NOTE: First test for 2D is to show that the minimum summed Wasserstein along the model axes is more that when using the known principle axes. If this doesn't work, need to rethink theory...

NOTE: how are we going to work out the marginal distributions from unevenly sampled posterior? Weighting by area of nearest neightbour cells would probably work but how computationally intense is this? Would surely be quicker than the posterior evaluation step, is purely based on model candidates and not the forward model - geometry problem.

NOTE: 1D solution is dependent on CDFs so might be able to deal with projections without having to use radial basis functions or similar - think of it as mass of regions rather than mass are particular points. See diagram in Peyre and Cuturi